<a href="https://colab.research.google.com/github/sophryu99/embedding-word2vec/blob/main/word2vec/word2vec_SGNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec - skip-gram with negative sampling, SGNS	
### **Skip-gram**
**Skip-gram**: 중심 단어로부터 주변 단어를 예측하는 모델 \
<img width="250" alt="Screen Shot 2021-05-08 at 3 25 24 PM" src="https://user-images.githubusercontent.com/46921003/117529256-9ecb7a80-b011-11eb-913c-a63d91643057.png"> <br/>
> 중심단어 cat을 사용해서 주변단어 the, fat, sat, on을 예측한다.

<br/>

### **Skip-gram with Negative Sampling, SGNS**: 

skip-gram의 문제점: 학습 과정에서는 '나무'에서 '열매'라는 단어를 예측할 때 매우 무관한 단어들(ex. 인형, 노트북)까지에 대해서도 학습을 해야하는 상황이 있다. 이러한 "**무관한 단어들에 대해서는 weight를 업데이트하지 않아도 된다**"가 negative sampling의 기본 아이디어이다.

Negative sampling은 Target이 negative인 단어들을 샘플링하는 과정이다. 
위의 인형이나 노트북과 같은 무관한 단어는 target 값이 0인 negative 값이고 "열매"는 target 값이 1인 positive 값이다. 

- 현재 문장에 없는 N개의 단어를 전체 데이터 셋에서 추출한다. -> Negative Samples
- 이렇게 추출한 Negative samples를 positive 값과 함께 학습한다.
- 네거티브 샘플링의 목적은 타겟 단어와 연관성이 없을 것이라고 추정되는 단어를 뽑는 것이다.

 
중심 단어와 주변 단어가 모두 입력이 되고, 이 두 단어가 실제로 윈도우 크기 내에 존재하는 이웃 관계인지 그 확률을 예측 <br/>
<img width="250" alt="Screen Shot 2021-05-08 at 3 34 50 PM" src="https://user-images.githubusercontent.com/46921003/117529516-ef8fa300-b012-11eb-8e70-045e65ebfedb.png"> <img width="213" alt="Screen Shot 2021-05-10 at 2 30 52 PM" src="https://user-images.githubusercontent.com/46921003/117610221-5391a300-b19c-11eb-946d-1c628da41f94.png">
<br/>

- wi 는 i번째 단어이고, f(wi)는 해당 단어의 빈도, (출현횟수/전체corpus수) 이다. 
- P(wi)는 wi가 나타날 확률 
<br/>

a. 기존의 **skip-gram 데이터셋**을 **SGNS의 데이터셋**으로 바꿔준다: <br/>
<img width="500" alt="Screen Shot 2021-05-08 at 3 37 10 PM" src="https://user-images.githubusercontent.com/46921003/117529585-4301f100-b013-11eb-87b0-c9938a4d4a1c.png"> <br/>

- 좌측의 테이블: 기존의 skip-gram을 학습하기 위한 데이터셋
  - 중심 단어를 입력, 주변 단어를 레이블로 설정
- 우측의 테이블: SGNS를 학습하기 위해서는 해당 테이블과 같이 수정해야함
  - 왼쪽의 데이터셋에서 중심단어와 주변단어를 각각 입력1, 입력2로 받아줌
  - 윈도우 내에서 이웃 관계였기에 레이블을 1로 설정해줌
  - 이웃관계가 아닌 단어(입력2)는 레이블을 0으로 설정해줌

b. **레이블이 0인 샘플**들을 준비한다. <br/>
<img width="693" alt="Screen Shot 2021-05-10 at 4 19 23 PM" src="https://user-images.githubusercontent.com/46921003/117620500-7d05fb00-b1ab-11eb-86c7-b1ec3f0d1d52.png"> <br/>
- 이제 이 데이터셋은 입력1과 입력2가 실제로 윈도우 크기 내에서 이웃 관계인 경우에는 레이블이 1, 아닌 경우에는 레이블이 0인 데이터셋이 된다.



## **1. SGNS 구현**

1. **두개의 임베딩 테이블 생성** <br/>
<img width="691" alt="Screen Shot 2021-05-08 at 3 56 07 PM" src="https://user-images.githubusercontent.com/46921003/117530027-e94ef600-b015-11eb-92f2-05b2b3a3961a.png">
- 좌측의 테이블: 중심단어(입력1)의 테이블 룩업을 위한 임베딩 테이블
- 우측의 테이블: 주변단어(입력2)의 테이블 룩업을 위한 임베딩 테이블
- 두개의 임베딩 테이블의 디멘션은 같다!

2. **중심단어와 주변단어의 임베딩 벡터값을 업데이트** <br/>

<img width="669" alt="Screen Shot 2021-05-08 at 3 58 39 PM" src="https://user-images.githubusercontent.com/46921003/117530101-421e8e80-b016-11eb-8b10-aa85f750f8a1.png"> <br/>

- 중심 단어와 주변단어의 내적값 `(L = log sigmoid (Xw.T * θv) + ∑neg(v) [log sigmoid (-Xw.T * θneg(v))]`을 이 모델의 예측값으로 하고, **레이블과의 오차로부터 역전파**하여 **중심 단어**와 **주변 단어**의 **임베딩 벡터값을 업데이트** 한다. 이후에는 선택적으로 두 개의 임베딩 테이블 중 좌측의 테이블을 최종 임베딩 테이블로 사용한다.

In [7]:
# SGNS Model Implementation
import torch
import torch.nn as nn
import torch.nn.functional as F

class SkipGramModel(nn.Module):
  """ 1.두개의 임베딩 테이블 생성, 및 임베딩 사이즈/디멘션 설정 """
  def __init__(self, emb_size, emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.w_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)  # 중심 단어용 임베딩 테이블
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)  # 주변 단어용 임베딩 테이블
        self._init_emb()

  def _init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.w_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)

  """ 2. 중심단어와 주변단어의 임베딩 벡터값을 업데이트
  내적의 결과는 1 또는 0을 예측하기 위해서 시그모이드 함수를 활성화 함수로 거쳐 최종 예측값을 얻는다.
  L = log sigmoid (Xw.T * θv) + ∑neg(v) [log sigmoid (-Xw.T * θneg(v))]
  """
  def forward(self, pos_w, pos_v, neg_v):
        emb_w = self.w_embeddings(torch.LongTensor(pos_w))  # tensor로 convert: [ mini_batch_size * emb_dimension ]
        emb_v = self.v_embeddings(torch.LongTensor(pos_v))
        neg_emb_v = self.v_embeddings(torch.LongTensor(neg_v))  # 변환된 크기: [ negative_sampling_number * mini_batch_size * emb_dimension ]
        score = torch.mul(emb_w, emb_v).squeeze()
        score = torch.sum(score, dim=1)
        score = F.logsigmoid(score)
        neg_score = torch.bmm(neg_emb_v, emb_v.unsqueeze(2)).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score) # L = log sigmoid (Xw.T * θv) + ∑neg(v) [log sigmoid (-Xw.T * θneg(v))]
        #print("Negative_score is:", neg_score[0][0].item())
        loss = -1 * (torch.sum(score) + torch.sum(neg_score))
        #print("Loss is", loss.item())
        return loss

  """ 3. 임베딩 결과 저장 """
  def save_embedding(self, id2word, file_name):
        embedding = self.w_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))

def test():
    model = SkipGramModel(100, 10)
    id2word = dict()
    for i in range(100):
        id2word[i] = str(i)
    pos_w = [0, 0, 1, 1, 1]
    pos_v = [1, 2, 0, 2, 3]
    neg_v = [[23, 42, 32], [32, 24, 53], [32, 24, 53], [32, 24, 53], [32, 24, 53]]
    model.forward(pos_w, pos_v, neg_v)
    model.save_embedding(id2word, 'test.txt')


if __name__ == '__main__':
    test()

## **2. SGNS Input Data 형식**
- (apples, pears) — real pair
- (apples, random word 1) — negative pair
- (apples, random word 2) — negative pair


In [28]:
# Data 준비
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('총 샘플 수 :',len(documents))

# 불필요한 토큰 제거 및 소문자화를 통한 정규화
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
# Null 값 처리
news_df.dropna(inplace=True)

# 불용어를 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()

# 단어가 1개 이하인 샘플의 인덱스를 찾아서 저장하고, 해당 샘플들은 제거.
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <= 1]
tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print('총 샘플 수 :',len(tokenized_doc))

# 단어 집합을 생성하고, 정수 인코딩 진행
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
총 샘플 수 : 11314


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


총 샘플 수 : 10940


In [29]:
# 20뉴스그룹 데이터 전처리 거친 후 파일로 저장
with open('./input_data.txt', 'w') as f:
    f.write(news_df['clean_doc'].str.cat(sep='\n'))

In [10]:
data = open('./input_data.txt')
data

<_io.TextIOWrapper name='./input_data.txt' mode='r' encoding='UTF-8'>

In [30]:
# Input data formatting
import numpy as np
from collections import deque

class InputData:
    def __init__(self, input_file_name, min_count):
        self.input_file_name = input_file_name
        self.input_file = open(self.input_file_name)  # 데이터 불러오기
        self.min_count = min_count  # 제거할 단어의 최소 빈도수
        self.wordId_frequency_dict = dict()  # 단어 ID- 빈도수 dict
        self.word_count = 0  # 단어 수 (반복되는 단어는 1로만 계산)
        self.word_count_sum = 0  # 총 단어 수 (반복되는 단어 수도 누적 됨)
        self.sentence_count = 0  # 문장 수
        self.id2word_dict = dict()  # 단어 ID - 단어 dict
        self.word2id_dict = dict()  # 단어 ID dict
        self._init_dict()  # dict 초기화
        self.sample_table = []
        self._init_sample_table()  # sample table 초기화
        self.word_pairs_queue = deque() # deque에 pair 저장
        # Training 진행 시 아웃풋
        print('Word Count is:', self.word_count)
        print('Word Count Sum is', self.word_count_sum)
        print('Sentence Count is:', self.sentence_count)

        print("===================================================")
        print('Get batch pairs (first 5):', self.get_batch_pairs(64, 4)[0:5])
        positive_batch = self.get_batch_pairs(64, 4)
        print('Get negative sampling (first 5):', self.get_negative_sampling(positive_batch, 4)[0:5])
        print('Length of positive_batch:', len(positive_batch))
        print('Length of negative_batch:', len(self.get_negative_sampling(positive_batch, 4)))

    def _init_dict(self):
        word_freq = dict()
        # word_frequency
        for line in self.input_file:
            line = line.strip().split(' ')  # 공백 제거
            self.word_count_sum += len(line)
            self.sentence_count += 1
            for word in line:
                try:
                    word_freq[word] += 1
                except:
                    word_freq[word] = 1
        word_id = 0
        # word2id_dict,id2word_dict, wordId_frequency_dict 초기화
        for per_word, per_count in word_freq.items():
            if per_count < self.min_count:  # 미니멈 단어 빈도수에 못 미치는 단어들 제거
                self.word_count_sum -= per_count
                continue
            self.id2word_dict[word_id] = per_word
            self.word2id_dict[per_word] = word_id
            self.wordId_frequency_dict[word_id] = per_count
            word_id += 1
        self.word_count = len(self.word2id_dict)

    def _init_sample_table(self):
        sample_table_size = 1e8
        pow_frequency = np.array(list(self.wordId_frequency_dict.values())) ** 0.75  # 단어 등장 빈도수 3/4
        word_pow_sum = sum(pow_frequency)  # 모든 단어의 총 단어 빈도
        ratio_array = pow_frequency / word_pow_sum  # 단어 빈도수 비율
        word_count_list = np.round(ratio_array * sample_table_size)
        for word_index, word_freq in enumerate(word_count_list):
            self.sample_table += [word_index] * int(word_freq)  # 목록 생성, 내용은 각 단어의 ID, 목록의 각 ID는 여러 번 반복되며, 반복 횟수는 매핑 후 단어 빈도입니다.
        self.sample_table = np.array(self.sample_table)

    # 미니 배치 크기의 양의 샘플 pair (w, v) 
    # w: 중심단어 ID v: 주변단어 ID
    # Window의 크기대로 샘플 pair 생성
    def get_batch_pairs(self, batch_size, window_size):
        while len(self.word_pairs_queue) < batch_size:
            for _ in range(10000): 
                self.input_file = open(self.input_file_name, encoding="utf-8")
                sentence = self.input_file.readline()
                if sentence is None or sentence == '':
                    continue
                wordId_list = []  # 문장의 모든 단어에 해당하는 ID
                for word in sentence.strip().split(' '):
                    try:
                        word_id = self.word2id_dict[word]
                        wordId_list.append(word_id)
                    except:
                        continue
                # Positive 샘플링 pair (w, v)을 찾고 Positive 샘플링 queue 에 추가
                for i, wordId_w in enumerate(wordId_list):
                    for j, wordId_v in enumerate(wordId_list[max(i - window_size, 0):i + window_size + 1]):
                        assert wordId_w < self.word_count
                        assert wordId_v < self.word_count
                        if i == j:  
                            continue
                        self.word_pairs_queue.append((wordId_w, wordId_v))
        result_pairs = []
        for _ in range(batch_size):
            result_pairs.append(self.word_pairs_queue.popleft())
        return result_pairs

    # Negative Sample 가져 오기 
    # Positive Sample 배열에서 필요한 Negative Sample의 갯수를 설정하고, 랜덤하게 Negative sample을 가져온다.
    def get_negative_sampling(self, positive_pairs, neg_count):
        neg_v = np.random.choice(self.sample_table, size=(len(positive_pairs), neg_count)).tolist()
        return neg_v

    # 배치를 설정하는 데 사용되는 데이터에서 양의 샘플링 로그를 추정
    def evaluate_pairs_count(self, window_size):
        return self.word_count_sum * (2 * window_size - 1) - (self.sentence_count - 1) * (1 + window_size) * window_size

In [31]:
if __name__ == '__main__':
    input_data = InputData('./input_data.txt', 3)
    input_data

Word Count is: 24788
Word Count Sum is 1128329
Sentence Count is: 11314
Get batch pairs (first 5): [(0, 1), (0, 2), (0, 3), (0, 4), (1, 0)]
Get negative sampling (first 5): [[3184, 5325, 1903, 3285], [18577, 16086, 2681, 1727], [156, 253, 2367, 5674], [28, 13536, 1105, 20844], [12130, 114, 1127, 14491]]
Length of positive_batch: 64
Length of negative_batch: 64


## **3. Word2Vec - SGNS 구현**

In [ ]:
# Word2Vec
import torch.optim as optim
from tqdm import tqdm

# hyper parameters
WINDOW_SIZE = 4  # window size
BATCH_SIZE = 64  # mini-batch
MIN_COUNT = 3  # 제거해야하는 저빈도 단어의 빈도
EMB_DIMENSION = 100  # embedding dimension
LR = 0.02  # 학습률
NEG_COUNT = 4 

class Word2Vec:
    def __init__(self, input_file_name, output_file_name):
        self.output_file_name = output_file_name
        self.data = InputData(input_file_name, MIN_COUNT)
        self.model = SkipGramModel(self.data.word_count, EMB_DIMENSION)
        self.lr = LR
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.lr)

    def train(self):
        print("SkipGram Training......")
        pairs_count = self.data.evaluate_pairs_count(WINDOW_SIZE)
        print("pairs_count", pairs_count)
        batch_count = pairs_count / BATCH_SIZE
        print("batch_count", batch_count)
        process_bar = tqdm(range(int(batch_count)))
        for i in process_bar:
            pos_pairs = self.data.get_batch_pairs(BATCH_SIZE, WINDOW_SIZE)
            pos_w = [int(pair[0]) for pair in pos_pairs]  # 중심단어
            pos_v = [int(pair[1]) for pair in pos_pairs]  # 주변단어
            # positive pairs에서 필요한 negative samples를 neg_v에 넣어주기
            neg_v = self.data.get_negative_sampling(pos_pairs, NEG_COUNT)

            # Loss calculation
            self.optimizer.zero_grad()
            loss = self.model.forward(pos_w, pos_v, neg_v)
            loss.backward()
            self.optimizer.step()

            if i * BATCH_SIZE % 100000 == 0:
                self.lr = self.lr * (1.0 - 1.0 * i / batch_count)
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = self.lr

        self.model.save_embedding(self.data.id2word_dict, self.output_file_name)


if __name__ == '__main__':
    w2v = Word2Vec(input_file_name='./input_data.txt', output_file_name="word_embedding.txt")
    w2v.train()




  0%|          | 0/119875 [00:00<?, ?it/s]

SkipGram Training......
pairs_count 7672043
batch_count 119875.671875


Streaming output truncated to the last 5000 lines.
 24%|██▍       | 29020/119875 [01:12<03:52, 390.22it/s]


 24%|██▍       | 29062/119875 [01:12<03:48, 397.86it/s]


 24%|██▍       | 29103/119875 [01:12<03:46, 400.48it/s]


 24%|██▍       | 29149/119875 [01:12<03:37, 416.42it/s]


 24%|██▍       | 29192/119875 [01:12<03:35, 420.28it/s]


 24%|██▍       | 29235/119875 [01:12<03:35, 421.11it/s]


 24%|██▍       | 29278/119875 [01:12<03:43, 406.04it/s]


 24%|██▍       | 29321/119875 [01:13<03:40, 410.93it/s]


 24%|██▍       | 29365/119875 [01:13<03:36, 417.42it/s]


 25%|██▍       | 29409/119875 [01:13<03:33, 422.74it/s]


 25%|██▍       | 29452/119875 [01:13<03:39, 411.84it/s]


 25%|██▍       | 29494/119875 [01:13<03:38, 413.66it/s]


 25%|██▍       | 29536/119875 [01:13<03:39, 411.18it/s]


 25%|██▍       | 29578/119875 [01:13<03:39, 412.22it/s]


 25%|██▍       | 29620/119875 [01:13<03:45, 401.01it/s]


 25%|██▍       | 29661/119875 [01:13<03:44, 401.14it/s]


 25%|██▍       | 2970